In [51]:
# import libraries and read in data
import pandas as pd
from IPython.display import display, HTML

# BookCorpus, from directory downloaded from https://yknzhu.wixsite.com/mbweb
bc_books = pd.read_csv('data/BookCorpus/books_in_bookcorpus.csv')

# BookCorpusOpen, downloaded from https://huggingface.co/datasets/bookcorpusopen
bcopen_books = pd.read_csv('data/BookCorpusOpen/2020-08-27-epub_urls.txt',header=None,names=['EpubLink'])

# Smashwords, scraped from smashwords.com in April 2021
sw_thru14 = pd.read_csv('data/Smashwords/smashwords_thru_2014.csv')
sw_after14 = pd.read_csv('data/Smashwords/smashwords_after_2014.csv')
sw_books = pd.concat([sw_thru14,sw_after14])

In [52]:
# merge data for clearer processing
bcopen_books['smashwords_id'] = bcopen_books.EpubLink.str.split('/',expand=True)[5]
bcopen_books['in_bcopen'] = True

sw_books['smashwords_id'] = sw_books.Link.str.split('/',expand=True)[5]
sw_books = sw_books.merge(bcopen_books, how='outer',on='smashwords_id')
sw_books.fillna(value='',inplace=True)

## General Composition

In [20]:
# How many instances (books) are there in total?
print(len(bc_books))

11040


In [63]:
# To align with the 11,038 reported by Zhu and Kiros et al., remove "adventure-all.txt" and "romance-all.txt"
extra_books = bc_books[(bc_books.fname.str.contains('adventure-all.txt')|bc_books.fname.str.contains('romance-all.txt'))]
bc_books = bc_books[~(bc_books.fname.str.contains('adventure-all.txt')|bc_books.fname.str.contains('romance-all.txt'))]

In [64]:
len(bc_books)

11038

In [66]:
# How many unique books are there in total?
# Base this on the fingerprint of each entry: filename + word count + disk usage
bc_books_unique=bc_books.drop_duplicates(subset=['fname','word_count','disk_usage'])
len(bc_books_unique)

7189

In [67]:
# How many words in the files?
bc_books.word_count.sum()

811601031

In [68]:
# Zhu and Kiros et al. report 984,846,357, so this does not align
# For now, we hypothesize that this is due to empty text files
# e.g. 'All_I_Want_for_Christmas_Is_a_Vampire.txt' is empty in the download from https://yknzhu.wixsite.com/mbweb

# If we add the words from the "extra books," the word count exceeds the original reported
extra_books.word_count.sum() + bc_books.word_count.sum()

1042409939

## Duplicate Listings

In [70]:
# How many listings may be duplicated?
# (based on the filename + word count + disk usage fingerprint)
len(bc_books) - len(bc_books_unique)

3849

In [59]:
# Look for filenames occuring multiple times
potential_dups = pd.DataFrame(bc_books.fname.value_counts())
potential_dups.columns = ['n']
potential_dups=potential_dups[potential_dups.n>1]

# how many file names occur multiple times?
print(len(potential_dups))

2930


In [62]:
# for now, remove filenames with unknown name scheme (e.g. 'u3041.txt', 'et4358.txt', 'b5096.txt')
named_dups=potential_dups[~potential_dups.index.str.match(r'(u|et|b)')]

# inspect 5 random filenames occuring multiple times, inspect listings
for index, row in named_dups.sample(5).iterrows():
    print("File name: {}".format(index))
    print("{} Occurrences:".format(len(bc_books[bc_books.fname==index])))
    display(HTML(bc_books[bc_books.fname==index].to_html()))

File name: 375455.txt
2 Occurrences:


,location,category,fname,word_count,disk_usage
4018,./Romance/375455.txt,Romance,375455.txt,55959,316K
4019,./Themes/375455.txt,Themes,375455.txt,55959,316K


File name: Darker_After_Midnight.txt
2 Occurrences:


,location,category,fname,word_count,disk_usage
9913,./Romance/Darker_After_Midnight.txt,Romance,Darker_After_Midnight.txt,120650,656K
9914,./Vampires/Darker_After_Midnight.txt,Vampires,Darker_After_Midnight.txt,120650,656K


File name: 513456.txt
2 Occurrences:


,location,category,fname,word_count,disk_usage
4356,./Fantasy/513456.txt,Fantasy,513456.txt,59597,336K
4357,./Science_fiction/513456.txt,Science_fiction,513456.txt,59597,336K


File name: 405841.txt
2 Occurrences:


,location,category,fname,word_count,disk_usage
6502,./Mystery/405841.txt,Mystery,405841.txt,79752,452K
6503,./Thriller/405841.txt,Thriller,405841.txt,79752,452K


File name: 452324.txt
2 Occurrences:


,location,category,fname,word_count,disk_usage
1247,./Adventure/452324.txt,Adventure,452324.txt,24334,148K
1248,./Science_fiction/452324.txt,Science_fiction,452324.txt,24334,148K


## Sample Comparisons

#### Genre Distribution

In [55]:
# Genre distribution in original BookCorpus
genre_df = pd.DataFrame(bc_books.category.value_counts())
genre_df.columns = ['BookCorpusN']
genre_df['BookCorpusP'] = (genre_df.BookCorpusN/genre_df.BookCorpusN.sum()*100).round(1).astype(str)+'%'
genre_df['BookCorpus'] = genre_df.BookCorpusP + ' (' + genre_df.BookCorpusN.astype(str) + ')'


# BookCorpusOpen
genre_df['BookCorpusOpenN'] = ''
for genre in genre_df.index.tolist():
    g=genre.replace('_',' ')
    genre_df.at[genre,'BookCorpusOpenN'] = len(sw_books[(sw_books.in_bcopen==True)&(sw_books.Categories.str.contains(g,case=False))])

total_bcopen = len(sw_books[(sw_books.in_bcopen==True)])
genre_df.BookCorpusOpenN = genre_df.BookCorpusOpenN.astype(int)
genre_df['BookCorpusOpenP'] = (genre_df.BookCorpusOpenN/total_bcopen*100).round(1).astype(str)+'%'
genre_df['BookCorpusOpen'] = genre_df.BookCorpusOpenP + ' (' + genre_df.BookCorpusOpenN.astype(str) + ')'



# Smashwords21
genre_df['Smashwords21'] = ''
for genre in genre_df.index.tolist():
    g=genre.replace('_',' ')
    genre_df.at[genre,'Smashwords21N'] = len(sw_books[sw_books.Categories.str.contains(g,case=False)])

    
total_smashwords = len(sw_books)
genre_df.Smashwords21N = genre_df.Smashwords21N.astype(int)
genre_df['Smashwords21P'] = (genre_df.Smashwords21N/total_smashwords*100).round(1).astype(str)+'%'
genre_df['Smashwords21'] = genre_df.Smashwords21P + ' (' + genre_df.Smashwords21N.astype(str) + ')'



to_print = genre_df[['BookCorpus','BookCorpusOpen','Smashwords21']]
display(HTML(to_print.to_html()))

,BookCorpus,BookCorpusOpen,Smashwords21
Romance,26.1% (2881),18.1% (3266),16.2% (62775)
Fantasy,13.6% (1502),17.2% (3107),10.7% (41460)
Science_fiction,7.5% (823),13.2% (2385),7.8% (30421)
New_Adult,6.9% (766),1.0% (178),0.7% (2745)
Young_Adult,6.8% (748),9.4% (1693),4.6% (17953)
Thriller,5.9% (646),7.1% (1283),5.7% (22078)
Mystery,5.6% (621),5.2% (935),4.7% (18258)
Vampires,5.4% (600),0.0% (0),0.0% (0)
Horror,4.1% (448),3.8% (686),3.9% (15070)
Teen,3.9% (430),9.4% (1697),4.7% (18083)


#### Religious Viewpoint Distribution

In [72]:
# based on recommendations by Dhamala and Sun et al. https://dl.acm.org/doi/abs/10.1145/3442188.3445924
religion_list = ['Sikhism', 'Judaism', 'Islam', 'Hinduism', 'Christianity', 'Buddhism', 'Atheism']

religion_df = pd.DataFrame(columns=['BookCorpus','BookCorpusOpen','Smashwords21'],index=religion_list)

,BookCorpus,BookCorpusOpen,Smashwords21
Sikhism,NaN,NaN,NaN
Judaism,NaN,NaN,NaN
Islam,NaN,NaN,NaN
Hinduism,NaN,NaN,NaN
Christianity,NaN,NaN,NaN
Buddhism,NaN,NaN,NaN
Atheism,NaN,NaN,NaN


In [75]:
for religion in religion_list:
    religion_df.at[religion,'BookCorpusOpen'] = len(sw_books[(sw_books.in_bcopen==True)&(sw_books.Categories.str.contains(religion,case=False))])
    religion_df.at[religion,'Smashwords21'] = len(sw_books[sw_books.Categories.str.contains(religion,case=False)])
    
religion_df

,BookCorpus,BookCorpusOpen,Smashwords21
Sikhism,NaN,0,15
Judaism,NaN,18,348
Islam,NaN,236,1263
Hinduism,NaN,11,244
Christianity,NaN,162,2544
Buddhism,NaN,32,485
Atheism,NaN,18,163
